In [116]:
import pandas as pd
import re
from unicodedata import normalize
import numpy as np

#!pip install pdfplumber
import pdfplumber

import seaborn as sns
import matplotlib.pyplot as plt

In [118]:
import warnings
warnings.filterwarnings("ignore")

In [120]:
import logging
logging.getLogger("pdfminer").setLevel(logging.ERROR)

In [122]:
# https://agricultura.gencat.cat/web/.content/de_departament/de02_estadistiques_observatoris/20_observatori_agroalimentari_de_preus/03_preus/preus_majoristes/enllacos-documents/fitxers-binaris/Fruita_Horta_Mercabarna.pdf
ruta_pdf="Fruita_Horta_Mercabarna.pdf"

In [124]:
nombresColumna = ['YEAR', 'MES', 'PRODUCTO', 'PRECIO O']
df_mercabarna = pd.DataFrame(columns=nombresColumna)


Fallos del proceso:

· 'HORTALISSES' se agrega a varios nombres de productos
· falta traducción
· falta quitar acentos

In [127]:
def procesar_fila_linea(linea, year):
    partes = linea.strip().split()
    numero_re = re.compile(r"^\d+([.,]\d+)?$")

    # Buscar el índice donde empiezan los precios
    try:
        inicio_precios = next(i for i, parte in enumerate(partes) if numero_re.match(parte))
    except StopIteration:
        return None 

    producto = " ".join(partes[:inicio_precios])
    
    precios = []
    for p in partes[inicio_precios:]:
        if numero_re.match(p):
            precios.append(float(p.replace(",", ".")))
        else:
            precios.append(np.nan)
    if precios:
        precios = precios[:-1]

    if not precios or all(np.isnan(p) for p in precios):
        return None

    while len(precios) < 12:
        precios.append(np.nan)
    
    meses = ["ENERO", "FEBRERO", "MARZO", "ABRIL", "MAYO", "JUNIO", "JULIO",
         "AGOSTO", "SEPTIEMBRE", "OCTUBRE", "NOVIEMBRE", "DICIEMBRE"]

    filas = []
    for i, precio in enumerate(precios):
        if not np.isnan(precio):
            mes = meses[i] if i < len(meses) else f"Mes{i+1}"
            filas.append([year, mes, producto, precio])

    return filas

In [129]:
with pdfplumber.open(ruta_pdf) as pdf:
    for i, pagina in enumerate(pdf.pages):# i = página
        print(f"- Procesando página {i+1}/{len(pdf.pages)}", end="\r", flush=True)
        texto = pagina.extract_text()
        if 'mitjanes' in texto:  # ignorar medias de grupos anuales
            pass
        else:
            lineas = texto.split("\n")
            if lineas[0] == 'MERCABARNA': # pagina con datos
                #print(lineas[2])  Producte - Preu €/kg G e ner Febrer Març Abril Maig Juny Juliol Agost Setembre Octubre Novembre Desembre Mitjana Anual
                final = True
                posicion = 3
                while final:
                    if len(lineas) > max(1, posicion): # evitar un problema de Out of index
                        #print(lineas[1].split()[1],posicion)
                        filas = procesar_fila_linea( lineas[posicion], lineas[1].split()[1] )
                        df_mercabarna = pd.concat([df_mercabarna, pd.DataFrame(filas, columns=nombresColumna)], ignore_index=True)
                        #print( procesar_fila_linea( lineas[posicion], lineas[1].split()[1] ) )
                        if lineas[posicion].split()[0] == 'Gabinet':
                            final = False
                    else: # por el Out of index
                        final = False
                    posicion+=1

In [130]:
errores_en_productos = [
    "HORTALISSES",
    "SENSE DETERMINAR",
    "SENSE DETERM.",
    "SENSE DETER."]

def limpiar_nombres_producto(serie, palabras):
    for palabra in palabras:
        serie = serie.str.replace(palabra, "", regex=False)
    return serie.str.strip()
df_mercabarna["PRODUCTO"] = limpiar_nombres_producto(df_mercabarna["PRODUCTO"], errores_en_productos)

df_mercabarna.loc[df_mercabarna["PRODUCTO"].str.contains("XIRIVIA"), "PRODUCTO"] = "XIRIVIA"
df_mercabarna = df_mercabarna[df_mercabarna["PRODUCTO"] != "FRUITES SENSE DETERMIN."]

In [131]:
# df_mercabarna["PRODUCTO"].unique()
# https://www.softcatala.org/traductor/

traducciones = {
    "ALBERCOCS": "ALBARICOQUE",
    "ALVOCATS": "AGUACATE",
    "AMETLLES": "ALMENDRA",
    "ARANJA": "POMELO",
    "CAQUIS": "CAQUI",
    "CARAMBOLA": "CARAMBOLA",
    "CASTANYES": "CASTAÑA",
    "CIRERES": "CEREZA",
    "COCOS": "COCO",
    "CODONYS": "MEMBRILLO",
    "DÀTILS": "DÁTIL",
    "FIGUES": "HIGO",
    "FIGA DE MORO": "HIGO CHUMBO",
    "FRUITES SENSE DETERMIN.": "FRUTA SIN DETERMINAR",  # Aunque la eliminarás
    "GERDS": "FRAMBUESA",
    "GROSELLA": "GROSELLA",
    "KIWIS": "KIWI",
    "LITXIS": "LICHI",
    "LLIMA": "LIMA",
    "LLIMONA": "LIMÓN",
    "MADUIXA": "FRESA",
    "MADUIXOT": "FRESÓN",
    "MAGRANES": "GRANADA",
    "MANDARINA CLEMENTINA": "MANDARINA CLEMENTINA",
    "MANDARINA SATSUMA": "MANDARINA SATSUMA",
    "MANDARINES SENSE DETER.": "MANDARINA SIN DETERMINAR",
    "MANGOS": "MANGO",
    "MARACUIÀ": "FRUTA DE LA PASIÓN",
    "MELÓ GALIA": "MELÓN GALIA",
    "MELÓ GROC": "MELÓN AMARILLO",
    "MELÓ": "MELÓN",
    "MELÓ VERD \"PIEL DE SAPO\"": "MELÓN PIEL DE SAPO",
    "MORA": "MORA",
    "NASHIS": "NASHI",
    "NECTARINA POLPA BLANCA": "NECTARINA PULPA BLANCA",
    "NECTARINA POLPA GROGA": "NECTARINA PULPA AMARILLA",
    "NESPRES": "NÍSPERO",
    "NOUS": "NUEZ",
    "OLIVES": "ACEITUNA",
    "PAPAIES": "PAPAYA",
    "PERA BLANQUILLA": "PERA BLANQUILLA",
    "PERA D'ESTIU FRUITES": "PERA DE VERANO",
    "PERA D'HIVERN": "PERA DE INVIERNO",
    "PERA PRIMERENCA": "PERA TEMPRANA",
    "PINYA TROPICAL": "PIÑA TROPICAL",
    "PITAHAIA": "PITAHAYA",
    "PLÀTAN CANARI": "PLÁTANO CANARIO",
    "PLÀTANS IMPORTACIÓ": "PLÁTANO DE IMPORTACIÓN",
    "POMA BICOLOR": "MANZANA BICOLOR",
    "POMA GROGA GOLDEN": "MANZANA GOLDEN AMARILLA",
    "POMA VERDA GRANNY SMITH": "MANZANA VERDE GRANNY SMITH",
    "POMA VERMELLA": "MANZANA ROJA",
    "POMES": "MANZANA",
    "PRÉSSEC GROC": "MELOCOTÓN AMARILLO",
    "PRÉSSEC VERMELL P.BLANCA": "MELOCOTÓN ROJO PULPA BLANCA",
    "PRÉSSECS VERMELL POLPA GROGA": "MELOCOTÓN ROJO PULPA AMARILLA",
    "PRUNA": "CIRUELA",
    "RAÏM BLANC": "UVA BLANCA",
    "RAÏM NEGRE": "UVA NEGRA",
    "RAMBUTÀ": "RAMBUTÁN",
    "SÍNDRIA AMB GRANA": "SANDÍA CON SEMILLAS",
    "SÍNDRIA SENSE GRANA": "SANDÍA SIN SEMILLAS",
    "TARONJA": "NARANJA",
    "XIRIMOIES": "CHIRIMOYA",
    "ALBERGÍNIA LLARGA": "BERENJENA LARGA",
    "ALBERGÍNIA RODONA": "BERENJENA REDONDA",
    "ALLS SECS": "AJO SECO",
    "ALLS TENDRES": "AJO TIERNO",
    "APIS": "APIO",
    "BATATA": "BATATA",
    "BLEDES": "ACELGA",
    "BORRALLES": "BORRAJA",
    "BROCULI": "BRÓCOLI",
    "BRÒQUIL": "BRÓCOLI",
    "CALÇOTS": "CALÇOT",
    "CARBASSA": "CALABAZA",
    "CARBASSÓ": "CALABACÍN",
    "CARDS": "CARDO",
    "CARXOFA": "ALCACHOFA",
    "CEBA SECA": "CEBOLLA",
    "CEBA SECA FIGUERES": "CEBOLLA FIGUERES",
    "CEBA TENDRA": "CEBOLLA TIERNA",
    "COGOMBRE": "PEPINO",
    "COLIFLOR": "COLIFLOR",
    "COLS": "COL",
    "COLS BRUSEL·LES": "COL DE BRUSELAS",
    "COLS LOMBARDA": "COL LOMBARDA",
    "COLS XINA": "COL CHINA",
    "CRÉIXENS": "BERRO",
    "ENCIAM ICEBERG": "LECHUGA ICEBERG",
    "ENCIAM LLARG": "LECHUGA LARGA",
    "ENCIAM SENSE DETER.": "LECHUGA SIN DETERMINAR",
    "ENDÍVIES": "ENDIVIA",
    "ESCAROLA": "ESCAROLA",
    "ESPÀRREC BLANC": "ESPÁRRAGO BLANCO",
    "ESPÀRREC VERD": "ESPÁRRAGO VERDE",
    "ESPINAC": "ESPINACA",
    "FAVES": "HABA",
    "FONOLLS": "HINOJO",
    "JULIVERT": "PEREJIL",
    "MONGETA BOBI": "JUDÍA BOBI",
    "MONGETA FINA": "JUDÍA FINA",
    "MONGETA PERONA": "JUDÍA PERONA",
    "MONGETES": "JUDÍA",
    "NAPS": "NAVO",
    "PASTANAGA": "ZANAHORIA",
    "PATATA BLANCA QUALIT.MONA LISA": "PATATA",
    "PATATA MONA LISA": "PATATA",
    "PATATA PRIMERENCA": "PATATA TEMPRANA",
    "PATATA VERMELLA": "PATATA ROJA",
    "PEBROT LAMUYO": "PIMIENTO LAMUYO",
    "PEBROT VERD ITALIÀ": "PIMIENTO VERDE",
    "PEBROT VERMELL": "PIMIENTO ROJO",
    "PÈSOLS": "GUISANTE",
    "PORROS": "PUERRO",
    "RAVES": "RÁBANO",
    "REMOLATXA": "REMOLACHA",
    "TOMÀQUET MADUR": "TOMATE",
    "TOMÀQUET VERD": "TOMATE VERDE",
    "VERDURES ALTRES": "OTRA VERDURA",
    "XIRIVIA": "CHIRIVÍA",
    "AVELLANES": "AVELLANA",
    "FEIJOA": "FEIJOA"
}

In [132]:
df_mercabarna["PRODUCTO"] = df_mercabarna["PRODUCTO"].replace(traducciones)

In [133]:
#https://es.stackoverflow.com/questions/135707/c%C3%B3mo-puedo-reemplazar-las-letras-con-tildes-por-las-mismas-sin-tilde-pero-no-l
def acentos(texto):
  texto = re.sub(
        r"([^cn\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f]))|c(?!\u0327(?![\u0300-\u036f])))[\u0300-\u036f]+", r"\1",
        normalize( "NFD", texto), 0, re.I
    )
  s = normalize( 'NFC', texto)
  return texto
df_mercabarna['PRODUCTO'] = df_mercabarna['PRODUCTO'].apply(acentos)

In [134]:
df_mercabarna.reset_index(drop=True, inplace=True)
df_mercabarna.to_csv("df_mercabarna.csv", index=False)

In [135]:
df_mercabarna.head(4)

,YEAR,MES,PRODUCTO,PRECIO O
0,2025,ENERO,ALBARICOQUE,2.65
1,2025,FEBRERO,ALBARICOQUE,3.00
2,2025,ENERO,AGUACATE,3.89
3,2025,FEBRERO,AGUACATE,4.00
